# Decision Tree Classifier

- [Overview](https://www.analyticsvidhya.com/blog/2021/08/decision-tree-algorithm/#:~:text=A%20decision%20tree%20algorithm%20is,each%20node%20of%20the%20tree.)
- [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [algorithm](https://www.javatpoint.com/machine-learning-decision-tree-classification-algorithm)
- [deeper understanding](https://www.geeksforgeeks.org/decision-tree/)
- [gini impurity](https://www.learndatasci.com/glossary/gini-impurity/)
- [asm](https://www.tutorialspoint.com/what-is-attribute-selection-measures)

## Intuition

- A decision tree classifer splits the data into distinct groups based on certain conditions, much like how a tree branches out. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.
- A simplified example of this is a game of guess who. You ask a series of questions to narrow down the posssible options until we reach our best guess. 

### Important Terminology

- **Root Node**: It represents the entire population or sample and this further gets divided into two or more homogeneous sets. At this point, we haven't made any decisions yet.
- **Splitting**: It is a process of dividing a node into two or more sub-nodes. For example, does this person have bronw hair? If yes, we split into a group of people with brown hair and those without.
- **Decision Node**: When a sub-node splits into further sub-nodes, then it is called a decision node. These will be your inner nodes.
- **Leaf/Terminal Node**: Nodes do not split is called Leaf or Terminal node. These are your final predictions.
- **Pruning**: When we remove sub-nodes of a decision node, this process is called pruning. You can think of this as removing unnecessary questions. If we know the person has brown hair, we don't need to ask if they have blonde hair.
- **Branch/Sub-Tree**: A sub-section of the entire tree is called a branch or sub-tree. We have a branch of brown hair individuals, but within that branch we have a sub-tree of people with brown hair and blue eyes.
- **Parent and Child Node**: A node, which is divided into sub-nodes is called a parent node of sub-nodes whereas sub-nodes are the child of a parent node.

## Strength and Weaknesses

- **Strengths**
    - Simple to understand and interpret. Trees can be visualised.
    - Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. 
    - The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
    - Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable.
    - Able to handle multi-output problems.
    - Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model, the explanation for the results is typically difficult to understand.
    - Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
    - Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.
- **Weaknesses**  
    - Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
    - Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
    - The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree.
    - There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.

## Algorithm

- **Step 1**: Begin with the entire dataset. This means our root node contains the entire dataset.
- **Step 2**: Find the best feature to split the data. This is done by calculating the Gini impurity or entropy of each feature. The feature with the lowest impurity is chosen to split the data.  
    - The gini impurity is a measure of how often a randomly chosen element would be incorrectly classified. It is calculated by summing the probability of each item being chosen times the probability of a mistake in categorising that item.  
    - The formula for gini impurity is: 
        - $Gini = 1 - \sum_{i=1}^{n} p_i^2$
        - where $p_i$ is the probability of an item being classified into a particular category.
    - The entropy of a dataset is a measure of the amount of uncertainty in the dataset. The entropy of a dataset is zero when the dataset is completely homogeneous.
    - The formula for entropy is:
        - $Entropy = -\sum_{i=1}^{n} p_i \log_2(p_i)$
        - where $p_i$ is the probability of an item being classified into a particular category.
- **Step 3**: Split the data based on the best feature. This means we create a decision node and two or more leaf nodes.
- **Step 4**: Repeat steps 1-3 for each leaf node until we have a tree that is deep enough or we have reached a stopping condition. This could be a maximum depth, minimum number of samples, or a minimum impurity.


## Implemening from Scratch

In [3]:
import pandas as pd 
import numpy as np

class DecisionNode:
    def __init__(self, feature_i=None, threshold=None,
                 value=None, true_branch=None, false_branch=None):
        self.feature_i = feature_i
        self.threshold = threshold
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch

def divide_on_feature(X, feature_i, threshold):
    '''
    Creating a vectorized function that can be applied to an array that checks if x is greater than or equal to a threshold.
    The return of this function is a boolean array that can be used to index the original array.
    This function is used to split the dataset based on a feature and threshold.
    The threshold is given by the best split found by the get_best_feature function.
    This function is used to create the true and false branches of a node.
    '''
    split_func = np.vectorize(lambda x: x >= threshold)
    return split_func(X[:, feature_i])

'''
Impurity functions measure the dispersion of the target variable of the target variable y.
These functions can be called within subsets of the data at each node to determine the next split.
Remember, the goal is to split impurity or increase the homogenity(or purity) of the target variable.
Remember, purity means that the target variable has the same value for all samples in a node. Hence, a clean clasification can be made.
'''

def gini_impurity(y):
    # My numpy implementation of gini impurity
    total = len(y)
    _, counts = np.unique(y, return_counts=True)
    p = counts / total
    gini = 1 - np.sum(p**2)
    return gini

def entropy(y):
    total = len(y)
    _, counts = np.unique(y, return_counts=True)
    p = counts / total
    entropy = -np.sum(p * np.log2(p + 1e-9))
    return entropy

def variance(y):
    return np.var(y)


def get_best_feature(X, y, impurity_func):
    '''
    If there is no best feature, the function returns None. This is used to create a leaf node.
    '''
    best_gain = 0
    best_feature = None
    best_threshold = None

    current_impurity = impurity_func(y)

    # For each feature in the dataset
    for feature_i in range(X.shape[1]):
        # Get unique values in a sorted order
        thresholds = np.unique(X[:, feature_i])
        
        # For each unique value in the feature
        for threshold in thresholds:
            # Split the dataset based on the feature and value
            split_func = np.vectorize(lambda x: x >= threshold)
            y1 = y[split_func(X[:, feature_i])]
            y2 = y[~split_func(X[:, feature_i])]

            # Calculate the weighted impurity of the two splits
            p = float(len(y1)) / (len(y1) + len(y2))
            impurity = p * impurity_func(y1) + (1 - p) * impurity_func(y2)

            info_gain = current_impurity - impurity

            if info_gain > best_gain:
                best_gain = info_gain
                best_feature = feature_i
                best_threshold = threshold

    return best_feature, best_threshold

def build_tree(X, y, impurity_func, max_depth=np.inf, min_size=2, depth=0):
    '''
    Main function to build the tree. A mask is a boolean array that can be used to index the original array.
    Masks are used to split the dataset based on the best feature and threshold.
    '''
    
    best_feature, threshold = get_best_feature(X, y, impurity_func)
    
    if best_feature is None or depth == max_depth or len(y) <= min_size:
        return DecisionNode(value=np.mean(y))

    mask = divide_on_feature(X, best_feature, threshold)
    true_branch = build_tree(X[mask], y[mask], impurity_func, max_depth, min_size, depth + 1)
    false_branch = build_tree(X[~mask], y[~mask], impurity_func, max_depth, min_size, depth + 1)
    
    return DecisionNode(best_feature, threshold, true_branch=true_branch, false_branch=false_branch)


def predict(node, x):
    if node.value is not None:
        return node.value
    if x[node.feature_i] >= node.threshold:
        return predict(node.true_branch, x)
    else:
        return predict(node.false_branch, x)